# Observations
1. Males by more, but females and "others" buy more expensive items.
2. Ages 35-39 buy the most expensive items on average.
3. You should target 35-39 year old females and others. 
4. This could have been done is a pivot table.
5. We should analyse game titles by gender.
6. These games are inexpensive.

In [ ]:
# Dependencies and Setup
import pandas as pd
from IPython.display import HTML

#Set up styles to make some df's look better
styles = [
    dict(selector="th", props=[("font-size", "150%"),
                               ("text-align", "center")]),
    dict(selector="caption", props=[("caption-side", "bottom")])
]

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

# checking the data
purchase_data.head()

## Player Count


In [104]:
# Get the total number of unique players by getting the length 0f the series
playerCount = {'Total Players': [len(purchase_data['SN'].unique())]}


playerCount_df = pd.DataFrame(data=playerCount)
html = (playerCount_df.style.hide_index().set_table_styles(styles))
html

Total Players
576


## Purchasing Analysis (Total)

In [109]:
#Retrieve the Item ID, Item Name, and Item Price columns
item_sales_df = purchase_data[["Item ID", "Item Name", "Price"]]

#Run basic calculations to obtain numbers for unique items, average price, etc.
uniqueItems = len(item_sales_df['Item ID'].unique())
averagePrice = item_sales_df['Price'].mean()
purchaseCount = item_sales_df['Item ID'].count()
totalRevenue = item_sales_df['Price'].sum()
results_df = pd.DataFrame({'Number of Unique Items':[uniqueItems],'Average Price':[averagePrice], 'Number of Purchases':[purchaseCount], 'Total Revenue':[totalRevenue]})

results_df['Average Price'] = results_df['Average Price'].map('${:,.2f}'.format)
results_df['Total Revenue'] = results_df['Total Revenue'].map('${:,.2f}'.format)

# Used this to hide the index :)
html = (results_df.style.hide_index().set_table_styles(styles))
html

Number of Unique Items,Average Price,Number of Purchases,Total Revenue
183,$3.05,780,"$2,379.77"


## Gender Demographics

In [111]:
# Get just the gender data
user_gender_df = purchase_data[['SN', 'Gender']].copy().drop_duplicates(['SN','Gender'])

#Run basic calculations to obtain numbers for totals and percentages of males, females, and others
maleTotal = user_gender_df.Gender.str.contains("Male").sum()
femaleTotal = user_gender_df.Gender.str.contains("Female").sum()
otherTotal = totalPlayers - (maleTotal + femaleTotal)
malePercentage = (maleTotal/totalPlayers)
femalePercentage = (femaleTotal/totalPlayers) 
otherPercentage = (otherTotal/totalPlayers) 


# Initialise dataframe of lists. 
data = {'Total Count':[maleTotal, femaleTotal, otherTotal], 'Percentage of Players':["{0:.2%}".format(malePercentage), "{0:.2%}".format(femalePercentage), "{0:.2%}".format(otherPercentage)]} 
  
# Creates pandas DataFrame. 
gender_df = pd.DataFrame(data, index =['Male', 'Female', 'Other / Non-Disclosed']) 
  
# print the data 
gender_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


## Purchasing Analysis (Gender)

In [114]:
# Get a dataframe grouped by gender    
genderPurchases_df = purchase_data.groupby(["Gender"]).sum()


#Format the price column
genderPurchases_df['Total Purchase Value'] = genderPurchases_df['Price'].map('${:,.2f}'.format).rename(columns={'Price': 'Total Purchase Value'})

#genderPurchases_df
#Get a series of total purchases by gender and add to the dataframe as Purchase Count
genderPurchases_df["Purchase Count"] = purchase_data.groupby(["Gender"]).count()["Price"]

# Get a series of mean purchase price by gender and add to the dataframe 
genderPurchases_df["Average Purchase Price"] = purchase_data.groupby(["Gender"]).mean()["Price"].map("${:.2f}".format)

# Get a series of total purchase price by person and add to the dataframe 
genderPurchases_df["Avg Total Purchase per Person"] = genderPurchases_df['Price'] /genderPurchases_df["Purchase Count"]
# Format column/series to money
genderPurchases_df["Avg Total Purchase per Person"] = genderPurchases_df["Avg Total Purchase per Person"].map("${:.2f}".format)

# Drop unneeded colums for display
genderPurchases_df.drop(columns=['Age', 'Price','Item ID', 'Purchase ID' ], axis=1, inplace=True)

# Reorder the columns for display 
genderPurchases_df = genderPurchases_df[['Purchase Count','Average Purchase Price','Total Purchase Value','Avg Total Purchase per Person']]
genderPurchases_df


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$3.20
Male,652,$3.02,"$1,967.64",$3.02
Other / Non-Disclosed,15,$3.35,$50.19,$3.35


## Age Demographics

In [117]:
# Define bins 
bins = [0, 9, 14, 19, 24, 29, 34, 39, 999]
# Create labels for these bins
group_labels = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

#clean up the Dups and get just the columns needed
# copy the data to avoid warning and remove dup because this is sales data
age_df  = purchase_data.copy().drop_duplicates(['SN','Age'])
age_df["Age Group"] = pd.cut(age_df["Age"], bins, labels=group_labels)

# Create a GroupBy object based upon "Age Group"
age_group = age_df.groupby( [ "Age Group"] , as_index=False).count()
#age_group 
# Get just the columns we want and rename them
age_group = age_group[["Age Group", "SN"]].rename(columns={'SN': 'Total Count'}) 

# Get the Summation Columns
age_group["Percentage of Players"] =age_group['Total Count'] / age_group['Total Count'].sum()
age_group["Percentage of Players"] = age_group["Percentage of Players"].map("{0:.2%}".format)
html = (age_group.style.hide_index().set_table_styles(styles))
html

Age Group,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

In [121]:
# Keep the dups this time
age_df  = purchase_data.copy()

# Bin the data using original bins and lables
age_df["Age Group"] = pd.cut(age_df["Age"], bins, labels=group_labels)

# Create a GroupBy object based upon "Age Group"
age_df =age_df.groupby(["Age Group"])

# Get a series with the bin sizes
age_count = age_df.size()
# Get a series the prices - using mean to get the indiviual price - may not be mest if they change the price
age_avg_price = age_df["Price"].mean().map("${:.2f}".format)
# Get a series the sum prices in a series 
age_tot_value = age_df["Price"].sum().map("${:.2f}".format)

#Build a new dataframe/pivot table based on purchases by age group
age_demographics = pd.DataFrame({"Purchase Count":age_count,
                             "Average Purchase Price": age_avg_price,
                             "Total Purchase Value": age_tot_value})

#reorder the columns to make since 
age_demographics = age_demographics[["Purchase Count","Average Purchase Price", "Total Purchase Value" ]]


age_demographics

,Purchase Count,Average Purchase Price,Total Purchase Value
Age Group,,,
<10,23,$3.35,$77.13
10-14,28,$2.96,$82.78
15-19,136,$3.04,$412.89
20-24,365,$3.05,$1114.06
25-29,101,$2.90,$293.00
30-34,73,$2.93,$214.00
35-39,41,$3.60,$147.67
40+,13,$2.94,$38.24


## Top Spenders


In [125]:
# Get a list of purchase proces by screen name 
spenders_df  = purchase_data[["SN","Price"]]

# Groug them
spenders_gb = spenders_df.groupby(["SN"])

# Get a series that has the count of purchases by each screen name 
purchaseCount_s = spenders_df.groupby(["SN"]).count()["Price"]

# Get a series that has the average of purchase price by each screen name 
purchaseAverage_s = spenders_df.groupby(["SN"]).mean()["Price"].rename("Average Value")

# Get a series that has the total of the purchases by each screen name 
purchaseValue_s = spenders_df.groupby(["SN"]).sum()["Price"].rename("Purchase Value")

# Build a new data frame 
top_spenders  = pd.DataFrame({"Purchase Count": purchaseCount_s, 
                             "Total Purchase Value": purchaseValue_s, 
                             "Average Purchase Value": purchaseAverage_s})

# Sort the data and format the dollar values 
top_spenders =top_spenders.sort_values('Total Purchase Value',  ascending=False)
top_spenders["Total Purchase Value"] = top_spenders["Total Purchase Value"].map("${:.2f}".format)
top_spenders["Average Purchase Value"] = top_spenders["Average Purchase Value"].map("${:.2f}".format)

# Show the top 5
top_spenders.head()

,Purchase Count,Total Purchase Value,Average Purchase Value
SN,,,
Lisosia93,5,$18.96,$3.79
Idastidru52,4,$15.45,$3.86
Chamjask73,3,$13.83,$4.61
Iral74,4,$13.62,$3.40
Iskadarya95,3,$13.10,$4.37


## Most Popular Items

In [129]:
# Get a dataframe of just the purchave info
raw_popular_items = purchase_data[["Item ID", "Item Name", "Price"]]

# Get a dataframe of the grouped by item of the sales count, price, and total in one swoop using groupby and agg
# note still using mean to get the sales price
popular_items = raw_popular_items.groupby(['Item ID', "Item Name"])['Price'].agg(['count','mean', 'sum'])

# Rename the columns for clarity
popular_items = popular_items.rename({'count': 'Purchase Count', 'mean': 'Item Price', 'sum':'Total Purchase Value' }, axis=1)

# Format the columns for display
popular_items["Item Price"] = popular_items["Item Price"].map("${:.2f}".format)
popular_items["Total Purchase Value"] = popular_items["Total Purchase Value"].map("${:.2f}".format)

# Order and show the data
# note this is  just ordered by sales count not sales value there are other items with sales the 8's
popular_items = popular_items.sort_values(by=['Purchase Count'],  ascending=False)
popular_items.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

In [103]:
# Mostly a rinse and repeat of the previous - just ordereing by totals sales value 
raw_popular_items = purchase_data[["Item ID", "Item Name", "Price"]]
popular_items = raw_popular_items.groupby(['Item ID', "Item Name"])['Price'].agg(['count','mean', 'sum'])
popular_items = popular_items.rename({'count': 'Purchase Count', 'mean': 'Item Price', 'sum':'Total Purchase Value' }, axis=1)

# here is the order that shows the money makers
popular_items = popular_items.sort_values(by=['Total Purchase Value'],  ascending=False)
popular_items["Item Price"] = popular_items["Item Price"].map("${:.2f}".format)
popular_items["Total Purchase Value"] = popular_items["Total Purchase Value"].map("${:.2f}".format)

popular_items.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
